<a href="https://colab.research.google.com/github/rmadushani/Three_vasopressor_problem/blob/main/eICU/codes/eICU_Three_Vasopressors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###
# @author: R. W. M. A. Madushani
# Created on Dec 20, 2019
###

In [ ]:
# Imports for accessing eICU data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

# Import libraries
import os
import pandas as pd
import numpy as np

In [ ]:
auth.authenticate_user()

In [ ]:
project_id = 'sccm-datathon'
os.environ['GOOGLE_CLOUD_PROJECT'] = project_id

In [ ]:
# Read data from BigQuery into pandas dataframes.
def run_query(query):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

In [ ]:
# Extract relevant patient data
patient_dat_query = \
"""
SELECT patientHealthSystemStayID, patientUnitStayID, uniquepid, gender, age, ethnicity, unitType, hospitalDischargeLocation, hospitalDischargeStatus
FROM `physionet-data.eicu_crd.patient`
"""

patient_dat = run_query(patient_dat_query)

## We consider only 'MICU', and 'Med-Surg ICU'
patient_dat_new = patient_dat.loc[(patient_dat.unitType=='MICU') | (patient_dat.unitType=='Med-Surg ICU'),: ].copy()

## Convert age>89 strings to numeric to extract patients with age>=18
patient_dat_new['age_int'] = patient_dat_new['age'].apply(lambda x: 90 if x=='> 89' else (-999 if x=='' else int(x)))
## Filter patients with age >= 18
patient_dat_new = patient_dat_new.loc[patient_dat_new.age_int >= 18, :]

## Remove records with missing hospital discharge status i.e., mortality
patient_dat_new = patient_dat_new.loc[patient_dat_new.hospitalDischargeStatus!='', :]
patient_dat_new.head()

,patientHealthSystemStayID,patientUnitStayID,uniquepid,gender,age,ethnicity,unitType,hospitalDischargeLocation,hospitalDischargeStatus,age_int
0,844848,1137569,011-43764,Female,75,Other/Unknown,MICU,Death,Expired,75
1,2462651,3036927,030-57208,Male,51,Caucasian,Med-Surg ICU,Home,Alive,51
2,2481889,3058863,030-13634,Female,31,Caucasian,Med-Surg ICU,Home,Alive,31
3,2494105,3072720,030-53536,Male,58,Caucasian,Med-Surg ICU,Home,Alive,58
4,2496542,3075429,030-59276,Female,68,Caucasian,Med-Surg ICU,Death,Expired,68
